In [60]:
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import Bounds
from calibration.solver_calibration import Calibration

from models.jump_vasicek import JumpVasicek
from models.jump_cir import JumpCIR

In [61]:
initial_model_params = {
    "kappa": 0.05,
    "mu_r": 0.01,
    "sigma": 0.01,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [63]:
def get_yield_curve(day=1):
    yield_curve_full = pd.read_csv("data/yield_curve02132021.csv")
    yield_curve_list = []
    for i in range(day):    
        yield_curve = pd.DataFrame(yield_curve_full.iloc[i, 1:]/100).astype(float)
        yield_curve["Maturity"] = list(range(1, len(yield_curve) + 1))
        yield_curve.columns = ["Yield", "Maturity"]
        yield_curve["Cum. Yield"] = yield_curve["Yield"] * yield_curve["Maturity"]
        yield_curve["Price"] = np.exp(-yield_curve["Cum. Yield"])
        Ts = yield_curve["Maturity"].values
        yield_curve_list.append(yield_curve["Price"])
    return pd.concat(yield_curve_list, axis=1).values.T, Ts

In [64]:
def calibrate(model_class, day=1, h=10):
    model_params = copy(initial_model_params)
    model_params["h"] = h
    prices, Ts = get_yield_curve(day)
    model = model_class(model_params)
    optimize_args = ("kappa", "mu_r", "sigma", "gamma")
    bounds = Bounds([0.001, 0.001, 0.01, 0.01], [1, 0.2, 0.2, 0.2])
    if __name__ == "__main__":
        calibrator = Calibration(
            model_class=model_class, n=100, m=26, r0=0.001,
            model_params=model_params, optimize_args=optimize_args)
        return calibrator.calibrate(
            initial_values=(model_params["kappa"], model_params["mu_r"], model_params["sigma"], model_params["gamma"]),
            Ts=Ts,
            prices=prices,
            bounds=bounds
        )

In [ ]:
optimal_vas_2_1   = calibrate(JumpVasicek, day=2, h=1)
print(optimal_vas_2_1)

In [ ]:
optimal_vas_3_1   = calibrate(JumpVasicek, day=3, h=1)
print(optimal_vas_3_1)

In [ ]:
optimal_vas_2_10   = calibrate(JumpVasicek, day=2, h=10)
print(optimal_vas_2_10)

In [ ]:
optimal_vas_3_10   = calibrate(JumpVasicek, day=3, h=10)
print(optimal_vas_2_10)

In [9]:
jump_cir = JumpCIR(inital_model_params)
optimize_args = ("kappa", "mu_r", "sigma", "gamma")
bounds = Bounds([0.001, 0.001, 0.01, 0.01], [1, 0.2, 0.2, 0.2])

In [ ]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=JumpCIR, n=100, m=26, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_cir = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"], inital_model_params["gamma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [ ]:
optimal_cir